In [4]:
## Importing packages.
# import Pkg
# Pkg.add("Tables")
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames, Plots, Statistics

In [94]:
## Function to Load Data.
function load_data(fname)
    data = DataFrame(CSV.File(fname, header=false)) |> DataFrame;
    return data
end;

In [2]:
hospital_mapping = CSV.read("../data/HospitalMapping.csv", DataFrame, pool=true);
distance_mapping_full = CSV.read("../data/DistanceMapping_full.csv", DataFrame, pool=true);
time_mapping = CSV.read("../data/TimeMapping.csv", DataFrame, pool=true);
depot_capacity = CSV.read("../data/depot_capacity.csv", DataFrame, pool=true);
vehicle_type = CSV.read("../data/vehicle_type.csv", DataFrame, pool=true);
# depot_hospital_distances = CSV.read("../data/depot_hospital_distances.csv", DataFrame, pool=true);

LoadError: UndefVarError: CSV not defined

In [3]:
hospital_mapping = hospital_mapping[:,2:end]
distance_mapping = distance_mapping[:,2:end]
time_mapping = time_mapping[:,2:end]
depot_capacity = depot_capacity[:,2:end]
vehicle_type = vehicle_type[:,2:end]
# depot_hospital_distances = depot_hospital_distances[:,2:end]

LoadError: UndefVarError: hospital_mapping not defined

### Creating Patient Mapping from Hospital Mapping

In [97]:
patients_per_hospital = sum(eachcol(hospital_mapping))
patient_mapping = []

for index = 1:size(patients_per_hospital)[1]
    for patient = 1:patients_per_hospital[index]
         push!(patient_mapping, index)
    end
end

In [98]:
hospital_mapping

Row,1,2,3,4
,Int64,Int64,Int64,Int64
1,0,0,0,0
2,0,0,1,0
3,2,0,0,0
4,0,1,0,0


In [114]:
patient_hospital_mapping = []
max_transfers = maximum(Matrix(hospital_mapping))
for i=1:max_transfers
    transfers = sort(Tuple.(findall(>=(i), Matrix(hospital_mapping))))
    for item in transfers
        push!(patient_hospital_mapping, item)
    end
#    push!(patient_hospital_mapping, sort(Tuple.(findall(>=(i), Matrix(hospital_mapping))))[0])
end

In [115]:
patient_hospital_mapping

4-element Vector{Any}:
 (2, 3)
 (3, 1)
 (4, 2)
 (3, 1)

In [ ]:
Dict2 = Dict("a" => 1, "b" => 2, "c" => 3)

In [116]:
patient_origin = []
patient_dest = []
aa
for index = 1:size(patient_hospital_mapping)[1]
    push!(patient_origin, patient_hospital_mapping[index][1])
    push!(patient_dest, patient_hospital_mapping[index][2])
end

In [117]:
patient_origin

4-element Vector{Any}:
 2
 3
 4
 3

### Enrich the Distance Mapping

In [89]:
D = size(depot_capacity)[1]

5

In [90]:
hospital_origin_index = D.+patient_origin
hospital_dest_index = D.+patient_dest

4-element Vector{Int64}:
 22
 22
 11
 18

In [91]:
patient_all_distances = distance_mapping[D.+patient_origin,:];
size(patient_all_distances)

(4, 28)

In [92]:
(D+1)

6

In [66]:
## Putting all distance into one dataframe.
depot_all_distances = distance_mapping[1:D,:]
hospital_all_distances = distance_mapping[(D+1):end,:]
distance_mapping_full = copy(depot_all_distances)
append!(distance_mapping_full, hospital_all_distances)
# append!(distance_mapping_full, patient_all_distances)

Row,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,3784,4197,5503,5546,8313,4303,3990,10182,5546,10,8313,4303,3990,10182,3180,3936,3797,2876,6246,3378,11316,11127,3182,11207,10968,7069,7727,1537
2,3086,0,441,4196,4239,10257,539,226,5724,4239,310,10257,539,226,5724,5637,2629,6254,4184,4536,6676,5387,6340,1298,5644,5789,4479,6420,1622
3,3506,448,0,4630,4673,10692,559,518,5378,4673,744,10692,559,518,5378,6057,3063,6674,4619,4956,7096,5041,5994,1733,5298,5443,5091,5202,2042
4,5755,4354,4775,0,834,9991,4369,4326,5973,834,4106,9991,4369,4326,5973,9458,1708,10075,3593,11721,6952,7107,6918,4894,6998,6759,11537,3518,5033
5,5318,3917,4018,641,0,9652,3686,3889,5634,232,3669,9652,3686,3889,5634,9021,1385,9638,3039,11382,6533,6768,6579,4457,6659,6420,11198,3179,4596
6,3102,310,731,3948,3991,0,3784,4197,5503,5546,3930,8313,4303,3990,10182,3180,3936,3797,2876,6246,3378,11316,11127,3182,6246,3378,11316,11127,3182
7,8239,10264,10685,10875,10701,3086,0,441,4196,4239,310,10257,539,226,5724,5637,2629,6254,4184,4536,6676,5387,6340,1298,4536,6676,5387,6340,1298
8,3689,598,611,4410,4453,3506,448,0,4630,4673,744,10692,559,518,5378,6057,3063,6674,4619,4956,7096,5041,5994,1733,4956,7096,5041,5994,1733
9,3317,226,511,4168,4211,5755,4354,4775,0,834,4106,9991,4369,4326,5973,9458,1708,10075,3593,11721,6952,7107,6918,4894,11721,6952,7107,6918,4894


### Defining Constants

In [67]:
# for Depots: 1:D
# for Patients: D+1:D+P+1
# for Hospitals: D+P+1:D+P+H+1

D = size(depot_capacity)[1]
P = size(patient_origin)[1]
H = size(hospital_mapping)[1]
K = size(vehicle_type)[1]
L = size(vehicle_type)[2]

d_start = 1
d_end = D
h_start = D+1
h_end = D+H
p_start = D+H+1
p_end = D+H+P;

In [68]:
d_ind = [d_start:1:d_end;]
p_ind = [p_start:1:p_end;]
h_ind = [h_start:1:h_end;];

In [69]:
h_d_ind = [h_ind; d_ind];

### Building Model

In [70]:
p_start

30

In [71]:
p_end

33

In [72]:
h_end

29

In [73]:
append!([1:1:5;],[7:1:10;])

9-element Vector{Int64}:
  1
  2
  3
  4
  5
  7
  8
  9
 10

In [74]:
# # Build model.
# model = Model(Gurobi.Optimizer)
# # set_optimizer_attribute(model, "OutputFlag", solver_output)


# # Insert variable.
# # We create a huge number of variables because we map multiple locations with the same index.
# @variable(model, x[i=1:p_end, j=1:h_end, k=1:K], Bin)


# # Insert constraints.

# # The patient_mapping holds the mapping only for the patients and thus the indices change.
# @constraint(model, [i=p_start:p_end], sum(x[i,h_start+patient_dest[i-p_start+1]-1,k] for k=1:K) == 1)

# @constraint(model, [i=p_start:p_end], sum(sum(x[i,j,k] for j=1:h_end) for k=1:K) == 1) # cant go to any depot or any other hospital than the mapped one.

# @constraint(model, [i=p_start:p_end, j=1:h_end, k=1:K], x[i,j,k] - sum(x[j,l,k] for l=append!([1:1:j-1;],[j+1:1:h_end;])))

# # @constraint(model, [k=1:K], sum(sum(x[i,j,k] for j=h_start:h_end) for i=p_start:p_end) <= sum(sum(x[i,j,k] for j in p_start:p_end) for i=1:D))

# # @constraint(model, [k=1:K], sum(sum(x[i,j,k] for j=[p_start:p_end]) for i=1:D) - sum(sum(x[i,j,k] for j=[h_start:h_end]) for i=1:D) == 0)

# # @constraint(model, [i=1:D, k=1:K], sum(sum(x[j,i,k]*vehicle_type[k,l] for j=[h_start:h_end]) for k=1:K) <= depot_capacity[i,l])

# # @constraint(model, [i=1:D, k=1:K], sum(sum(x[i,j,k]*vehicle_type[k,l] for j=[p_start:p_end]) for k=1:K) <= sum(sum(x[i,j,k]*vehicle_type[k,l] for j=[p_start:p_end]) for k=1:K))

# ## 1. Total Waste Collected Constraint.


# # Objective.
# @objective(model, Min, 
#     sum(sum(sum(distance_mapping_full[i,j]*x[i,j,k] for k=1:K) for j=1:h_end) for i=1:p_end) # distance from patient to hospital
# #     sum(depot_hospital_distances[i,j]*x[i,j,k] for k=1:K)
# )


# # Optimize.
# optimize!(model)

In [78]:
# Build model.
model = Model(Gurobi.Optimizer)
# set_optimizer_attribute(model, "OutputFlag", solver_output)


# Insert variable.
# We create a huge number of variables because we map multiple locations with the same index.
@variable(model, x[i=1:h_end, j=1:h_end, k=1:K], Bin)
@variable(model, y[i=1:h_end, j=1:h_end, k=1:K], Bin)


# Insert constraints.

@constraint(model, [i=:], sum(x[hospital_origin_index[i], hospital_dest_index[i], k] for k=1:K) == 1)

@constraint(model, [k=1:K], x[i,j,k])

@constraint(model, [], )


# @constraint(model, [k=1:K], sum(sum(x[i,j,k] for j=1:h_end) for i=1:h_end) 

# @constraint(model, [i=p_start:p_end], sum(sum(x[i,j,k] for j=1:h_end) for k=1:K) == 1) # cant go to any depot or any other hospital than the mapped one.

# @constraint(model, [i=p_start:p_end, j=1:h_end, k=1:K], x[i,j,k] - sum(x[j,l,k] for l=append!([1:1:j-1;],[j+1:1:h_end;])))

# @constraint(model, [k=1:K], sum(sum(x[i,j,k] for j=h_start:h_end) for i=p_start:p_end) <= sum(sum(x[i,j,k] for j in p_start:p_end) for i=1:D))

# @constraint(model, [k=1:K], sum(sum(x[i,j,k] for j=[p_start:p_end]) for i=1:D) - sum(sum(x[i,j,k] for j=[h_start:h_end]) for i=1:D) == 0)

# @constraint(model, [i=1:D, k=1:K], sum(sum(x[j,i,k]*vehicle_type[k,l] for j=[h_start:h_end]) for k=1:K) <= depot_capacity[i,l])

# @constraint(model, [i=1:D, k=1:K], sum(sum(x[i,j,k]*vehicle_type[k,l] for j=[p_start:p_end]) for k=1:K) <= sum(sum(x[i,j,k]*vehicle_type[k,l] for j=[p_start:p_end]) for k=1:K))

## 1. Total Waste Collected Constraint.


# Objective.
@objective(model, Min, 
    sum(sum(sum(distance_mapping_full[i,j]*x[i,j,k] for k=1:K) for j=1:h_end) for i=1:h_end) # distance from patient to hospital
#     sum(depot_hospital_distances[i,j]*x[i,j,k] for k=1:K)
)


# Optimize.
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 3364 columns and 8 nonzeros
Model fingerprint: 0xe83d91e9
Variable types: 0 continuous, 3364 integer (3364 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 33998.000000
Presolve removed 4 rows and 3364 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 33998 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.399800000000e+04, best bound 3.399800000000e+04, gap 0.0000%

User-callback calls 111, time in user-callback

In [261]:
x_k1 = DataFrame(value.(x)[:,:,1], :auto)
x_k2 = DataFrame(value.(x)[:,:,2], :auto)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [263]:
sum(sum(x_k2[i,j] for j=1:h_end) for i=1:p_end)

7.0

In [264]:
x_k2

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [265]:
CSV.write("../data/x_k1.csv", x_k1)
CSV.write("../data/x_k2.csv", x_k2)

"../data/x_k2.csv"

In [68]:
println(d_start)
println(d_end)
println(p_start)
println(p_end)
println(h_start)
println(h_end)

1
5
6
9
10
33
